In [122]:
import pandas as pd
import requests
from config import api_key
import json

In [123]:
dem_candidate=pd.read_csv("dem_candidate.csv")
dem_finance=pd.read_csv("dem_finance_data.csv")

In [124]:
dem_candidate=dem_candidate.rename(columns={
    "state":"candidate_state"
})
dem_finance=dem_finance.rename(columns={
    "state":"donor_state"
})

In [125]:
dem_data=pd.DataFrame.merge(dem_candidate, dem_finance, on="candidate_id")

In [126]:
dem_data.head(1)

,Unnamed: 0_x,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,Unnamed: 0_y,donor_state,donation_count,total_raised,cycle
0,0,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],DEM,P,0,AZ,114,21611.44,2018


In [127]:
dem_data=dem_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
dem_data

,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised
0,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,AZ,114,21611.44
1,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,CA,5,600.00
2,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,IL,8,1800.85
3,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MA,3,1500.00
4,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MI,1,250.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1013,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,OH,4,200.00
1014,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,OT,1,100.00
1015,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,PA,12,290.00
1016,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,TX,2,500.00


In [134]:
# if candidate_state = donor_state, in_state=True, if not, then False

in_state=(dem_data.candidate_state==dem_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state.head()

,in_state
0,True
1,False
2,False
3,False
4,False


In [135]:

dem_data=pd.DataFrame.merge(dem_data,in_state, left_index=True, right_index=True)

In [136]:
dem_data=dem_data.rename(columns={
    0:"in_state"
})
dem_data.head()

,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised,in_state
0,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,AZ,114,21611.44,True
1,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,CA,5,600.00,False
2,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,IL,8,1800.85,False
3,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MA,3,1500.00,False
4,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MI,1,250.00,False


In [137]:
# for every candidate, add up donor_count, and total_raised, if in_state == False

aggregate=dem_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})

In [140]:
aggregate.head()

donation_count total_raised
                                 sum          sum
candidate_id in_state                            
H2MA04073    False              2298    842077.58
             True               2773    907995.49
S0AL00156    False             30509   3751847.85
             True               6465   1974748.72
S0CA00199    False               869    712400.58

In [88]:
aggregate